In [5]:
import pandas as pd
from datetime import datetime, timedelta

In [6]:
inputs_path = "inputs.xlsx"

In [7]:
params = pd.read_excel(inputs_path,skiprows=5,usecols='B,C')

d = pd.Series(params.iloc[:, 0].values, index=params.iloc[:, 1]).to_dict()
d

{'ENTRAXE': 2.5,
 'HAUTEUR': 3,
 'RAMPANT': 1.135,
 'TYPE_PANEL': 'False',
 'LARGEUR_BANDE': 0.18,
 'NB_BANDES': 4,
 'ANGLE_ORIENTATION': 13,
 'start_date': 2023,
 'HEURE_DEB': 6,
 'HEURE_FIN': 22,
 'TYPE_CULT': 1,
 'FINESSE': 1}

In [8]:
# Paramètres de configuration de la géométrie
######
GRID_SIZE = 50 # Taille du champ de panneau pour la simulation
ENTRAXE = d.get('ENTRAXE') # Espacement entre lignes de panneaux (= inter-rang)
HAUTEUR = d.get('HAUTEUR') # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
RAMPANT = d.get('RAMPANT') # Largeur du panneau
NB_RANGS = 10 # Nombre de panneaux par ligne
NB_PVP_RANGS = int(GRID_SIZE / ENTRAXE) # Nombre de lignes
LONGUEUR_PVP = GRID_SIZE / (NB_RANGS) # Longueur des panneaux dans une même ligne

column_spacing = ENTRAXE # Espacement entre lignes de panneaux (= inter-rang)
row_spacing = GRID_SIZE / NB_RANGS # Espacement entre panneaux d'une même ligne (même support)

TYPE_PANEL = bool(d.get('TYPE_PANEL'))
LARGEUR_BANDE = d.get('LARGEUR_BANDE')
NB_BANDES = int(d.get('NB_BANDES'))

ANGLE_ORIENTATION = d.get('ANGLE_ORIENTATION')  # Rotation globale autour de l'axe Z (Orientation parcelle)

# Paramètres annuels
######
start_date = datetime(year=int(d.get('start_date')), month=1, day=1, hour=6)  # Début de l'année (éviter les années bissextiles)
end_date = datetime(year=int(d.get('start_date'))+1, month=1, day=1, hour=6)  # Fin de l'année (début de l'année suivante)

# Paramètres horaires
######
if d.get('HEURE_DEB') is not None:
    HEURE_DEB = int(d.get('HEURE_DEB'))
else : 
    HEURE_DEB = 6
if d.get('HEURE_FIN') is not None:
    HEURE_FIN = int(d.get('HEURE_FIN'))
else :
    HEURE_FIN = 22
NB_HEURES_ANNEES = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days

# Paramètres radiance
#####
TYPE_CULT = d.get('TYPE_CULT')
if TYPE_CULT == 1 : #Vine
    HAUTEUR_MESURE = 1 # Hauteur de mesure dans IncidentRadiation, en m
elif TYPE_CULT == 2 : #Arboriculture
    HAUTEUR_MESURE = 2,5 # Hauteur de mesure dans IncidentRadiation, en m
elif TYPE_CULT == 3 : #Field Crop
    HAUTEUR_MESURE = 0.5

FINESSE = d.get('FINESSE') # Finesse de la fonction IncidentRadiation (1 = peu précis, 0 = très précis) 